Notebook dedicado a explorar métodos de lectura y análisis DF. Para ello se trabajara con un fichero del portal de datos abiertos del ayuntamiento de Madrid, especificamente el Padron de Madrid, los cuales han están almacenados en una tabla en formato Parquet.

Se hacen pruebas con los métodos orderBy, sort, where, filter, además de groupBy, cast de tipos

In [1]:
// Leemos Datos procedentes de tabla cuyos datos están en formato parquet
val sourceParquetDF = spark.read.parquet("./dataOutput/df/parquet").cache()

// obtenemos el total de registros
val totalRecords = sourceParquetDF.count()

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.36:4043
SparkContext available as 'sc' (version = 3.0.2, master = local[*], app id = local-1615758019669)
SparkSession available as 'spark'


sourceParquetDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]
totalRecords: Long = 237675


In [2]:
import org.apache.spark.sql.functions._

val totalBySexAndTypeDF = sourceParquetDF.withColumn("Hombres", col("EspanolesHombres") + col("ExtranjerosHombres"))
                                       .withColumn("Mujeres", col("EspanolesMujeres") + col("ExtranjerosMujeres"))

val distritoAndBarrioPeopleAggDF = totalBySexAndTypeDF.groupBy("DESC_DISTRITO", "DESC_BARRIO")
.agg(
  sum("EspanolesHombres").alias("TotalEspanolesHombres"),
  sum("EspanolesMujeres").alias("TotalEspanolesMujeres"),
  sum("ExtranjerosHombres").alias("TotalExtranjerosHombres"),
  sum("ExtranjerosMujeres").alias("TotalExtranjerosMujeres"),
  sum("Hombres").alias("TotalHombres"),
  sum("Mujeres").alias("TotalMujeres")
  )
.withColumn("TotalPersonas", col("TotalHombres") + col("TotalMujeres"))
.select("DESC_DISTRITO", "DESC_BARRIO", "TotalEspanolesHombres", "TotalEspanolesMujeres", "TotalExtranjerosHombres", "TotalExtranjerosMujeres", "TotalHombres", "TotalMujeres", "TotalPersonas").cache()

distritoAndBarrioPeopleAggDF.orderBy(col("TotalPersonas").desc).show(false)

+-------------------+--------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+
|DESC_DISTRITO      |DESC_BARRIO         |TotalEspanolesHombres|TotalEspanolesMujeres|TotalExtranjerosHombres|TotalExtranjerosMujeres|TotalHombres|TotalMujeres|TotalPersonas|
+-------------------+--------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+
|LATINA             |ALUCHE              |25257                |29932                |5592                   |6609                   |30849       |36541       |67390        |
|FUENCARRAL-EL PARDO|VALVERDE            |26976                |29226                |3730                   |4515                   |30706       |33741       |64447        |
|HORTALEZA          |VALDEFUENTES        |28206                |28615                |2824                   |3748           

import org.apache.spark.sql.functions._
totalBySexAndTypeDF: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 12 more fields]
distritoAndBarrioPeopleAggDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DESC_DISTRITO: string, DESC_BARRIO: string ... 7 more fields]


In [3]:
val distritoPeopleAggDF = distritoAndBarrioPeopleAggDF.groupBy("DESC_DISTRITO").agg(
sum("TotalEspanolesHombres").alias("TotalEspanolesHombres"),
  sum("TotalEspanolesMujeres").alias("TotalEspanolesMujeres"),
  sum("TotalExtranjerosHombres").alias("TotalExtranjerosHombres"),
  sum("TotalExtranjerosMujeres").alias("TotalExtranjerosMujeres"),
  sum("TotalHombres").alias("TotalHombres"),
  sum("TotalMujeres").alias("TotalMujeres"),
  sum("TotalPersonas").alias("TotalPersonas")
)
.withColumn("TotalEspanoles", col("TotalEspanolesHombres") + col("TotalEspanolesMujeres"))
.withColumn("TotalExtranjeros", col("TotalExtranjerosHombres") + col("TotalExtranjerosMujeres"))

distritoPeopleAggDF.orderBy(col("TotalPersonas").desc).show(false)

+-------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+--------------+----------------+
|DESC_DISTRITO      |TotalEspanolesHombres|TotalEspanolesMujeres|TotalExtranjerosHombres|TotalExtranjerosMujeres|TotalHombres|TotalMujeres|TotalPersonas|TotalEspanoles|TotalExtranjeros|
+-------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+--------------+----------------+
|CARABANCHEL        |94274                |108948               |27018                  |29908                  |121292      |138856      |260148       |203222        |56926           |
|FUENCARRAL-EL PARDO|107136               |118780               |10034                  |13021                  |117170      |131801      |248971       |225916        |23055           |
|LATINA             |92207                |106512               |19780

distritoPeopleAggDF: org.apache.spark.sql.DataFrame = [DESC_DISTRITO: string, TotalEspanolesHombres: bigint ... 8 more fields]


In [4]:
import org.apache.spark.sql.types.DecimalType

val distritoPeopleAggRatioDF = distritoPeopleAggDF
.withColumn("%_Espanoles", (lit(100) * col("TotalEspanoles") / col("TotalPersonas")).cast(DecimalType(4,2)))
.withColumn("%_Extranjeros", (lit(100) * col("TotalExtranjeros") / col("TotalPersonas")).cast(DecimalType(4,2)))
.withColumn("%_Mujeres", (lit(100) * col("TotalMujeres") / col("TotalPersonas")).cast(DecimalType(4,2)))
.withColumn("%_Hombres", (lit(100) * col("TotalHombres") / col("TotalPersonas")).cast(DecimalType(4,2)))

// EL método sort y orderBy arrojan el mismo resultado
// El último es simplemente un alias del primero
distritoPeopleAggRatioDF.sort(col("TotalPersonas").desc).show(false)

+-------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+--------------+----------------+-----------+-------------+---------+---------+
|DESC_DISTRITO      |TotalEspanolesHombres|TotalEspanolesMujeres|TotalExtranjerosHombres|TotalExtranjerosMujeres|TotalHombres|TotalMujeres|TotalPersonas|TotalEspanoles|TotalExtranjeros|%_Espanoles|%_Extranjeros|%_Mujeres|%_Hombres|
+-------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+--------------+----------------+-----------+-------------+---------+---------+
|CARABANCHEL        |94274                |108948               |27018                  |29908                  |121292      |138856      |260148       |203222        |56926           |78.12      |21.88        |53.38    |46.62    |
|FUENCARRAL-EL PARDO|107136               |118780               |10034  

import org.apache.spark.sql.types.DecimalType
distritoPeopleAggRatioDF: org.apache.spark.sql.DataFrame = [DESC_DISTRITO: string, TotalEspanolesHombres: bigint ... 12 more fields]


In [5]:
// EL método filter y where arrojan el mismo resultado
// El último es simplemente un alias del primero
distritoPeopleAggRatioDF.filter(col("TotalPersonas") > 100000).sort(col("TotalPersonas").desc).show(false)

+-------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+--------------+----------------+-----------+-------------+---------+---------+
|DESC_DISTRITO      |TotalEspanolesHombres|TotalEspanolesMujeres|TotalExtranjerosHombres|TotalExtranjerosMujeres|TotalHombres|TotalMujeres|TotalPersonas|TotalEspanoles|TotalExtranjeros|%_Espanoles|%_Extranjeros|%_Mujeres|%_Hombres|
+-------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+--------------+----------------+-----------+-------------+---------+---------+
|CARABANCHEL        |94274                |108948               |27018                  |29908                  |121292      |138856      |260148       |203222        |56926           |78.12      |21.88        |53.38    |46.62    |
|FUENCARRAL-EL PARDO|107136               |118780               |10034  